### Consume an API with Python

In [2]:
import requests

### The GET Request

In [3]:
response = requests.get('https://pyprogrammer.com/api/v1')
response.status_code

200

In [4]:
# response implements __bool__ methods 
# True for 200,.., 300,...
# False for 500,.. 400,...
bool(response)  # 200, 204, 304,...

True

In [5]:
# we can use raise for status to handle exceptions in our code
try:
    response = requests.get("https://pyprogrammer.com/api/v1")
    # If the response was successful, no Exception will be raised
    response.raise_for_status()
except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')  # Python 3.6
except Exception as err:
    print(f'Other error occurred: {err}')  # Python 3.6
else:
    print('Success!')

Success!


In [6]:
type(response.content)  # content access to raw bytes from json

bytes

In [19]:
type(response.text)  # this uses the default renderer

str

In [24]:
response.json()  # json.loads(response.text)

{'posts': 'http://pyprogrammer.com/api/v1/posts/',
 'categories': 'http://pyprogrammer.com/api/v1/categories/',
 'tags': 'http://pyprogrammer.com/api/v1/tags/',
 'products': 'http://pyprogrammer.com/api/v1/products/',
 'producttypes': 'http://pyprogrammer.com/api/v1/producttypes/'}

In [25]:
response.headers
response.headers.keys()

KeysView({'Server': 'nginx/1.14.0 (Ubuntu)', 'Date': 'Sun, 02 Feb 2020 09:53:47 GMT', 'Content-Type': 'application/json', 'Content-Length': '269', 'Connection': 'keep-alive', 'Vary': 'Accept, Cookie', 'Allow': 'GET, HEAD, OPTIONS', 'X-Frame-Options': 'SAMEORIGIN'})

### Query String Parameters

In [26]:
import requests

# Search GitHub's repositories for requests
response = requests.get(
    'https://pyprogrammer.com/api/v1/posts',
    params={'q': 'requests+language:python'},
)

# Inspect some attributes of the `requests` repository
json_response = response.json()
response.status_code

403

### POST, PATCH, PUT

In [61]:
requests.post('https://httpbin.org/post', data={'key':'value'})
requests.put('https://httpbin.org/put', data={'key':'value'})
requests.delete('https://httpbin.org/delete')
requests.head('https://httpbin.org/get')
requests.patch('https://httpbin.org/patch', data={'key':'value'})
requests.options('https://httpbin.org/get')

<Response [200]>

In [62]:
# POST, PUT and PATCH Differences

### Inspecting yopur requests

In [ ]:
response = requests.post('https://httpbin.org/post', json={'key':'value'})
response.request.headers['Content-Type']
response.request.url
response.request.body

b'{"key": "value"}'

### Authentication

In [ ]:
from getpass import getpass
requests.get('https://api.github.com/user', auth=('username', getpass()))

# is the same
from requests.auth import HTTPBasicAuth
from getpass import getpass
requests.get(
    'https://api.github.com/user',
    auth=HTTPBasicAuth('username', getpass())
)

In [ ]:
import requests
from requests.auth import AuthBase

class TokenAuth(AuthBase):
    """Implements a custom authentication scheme."""

    def __init__(self, token):
        self.token = token

    def __call__(self, r):
        """Attach an API token to a custom auth header."""
        r.headers['X-TokenAuth'] = f'{self.token}'  # Python 3.6+
        return r


requests.get('https://httpbin.org/get', auth=TokenAuth('12345abcde-token'))

### SSL Verification

In [ ]:
requests.get('https://api.github.com', verify=False)

### Performance

In [ ]:
requests.get('https://api.github.com', timeout=3.05)

In [ ]:
# If the request establishes a connection within 2 seconds and receives data within 5 seconds o

In [ ]:
requests.get('https://api.github.com', timeout=(2, 5))

In [ ]:
import requests
from requests.exceptions import Timeout

try:
    response = requests.get('https://api.github.com', timeout=1)
except Timeout:
    print('The request timed out')
else:
    print('The request did not time out')

In [ ]:
# sessions

In [ ]:
import requests
from getpass import getpass

# By using a context manager, you can ensure the resources used by
# the session will be released after use
with requests.Session() as session:
    session.auth = ('username', getpass())

    # Instead of requests.get(), you'll use session.get()
    response = session.get('https://api.github.com/user')

# You can inspect the response just like you did before
print(response.headers)
print(response.json())

In [ ]:
# max retries

In [ ]:
import requests
from requests.adapters import HTTPAdapter
from requests.exceptions import ConnectionError

github_adapter = HTTPAdapter(max_retries=3)

session = requests.Session()

# Use `github_adapter` for all requests to endpoints that start with this URL
session.mount('https://api.github.com', github_adapter)

try:
    session.get('https://api.github.com')
except ConnectionError as ce:
    print(ce)